In [24]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, callback, Output, Input

In [39]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, callback_context

app = Dash(__name__)

# -- Import and clean data (importing csv into pandas)
df = pd.read_csv("https://raw.githubusercontent.com/Coding-with-Adam/Dash-by-Plotly/master/Other/Dash_Introduction/intro_bees.csv")

df = df.groupby(['State', 'ANSI', 'Affected by', 'Year', 'state_code'])[['Pct of Colonies Impacted']].mean()
df.reset_index(inplace=True)

# Create the default map figure to match the initial state

dff = df.copy()
dff = dff[dff["Year"] == option_slctd]
dff = dff[dff["Affected by"] == "Varroa_mites"]

default_map_figure = px.choropleth(
            data_frame=dff,
            locationmode='USA-states',
            locations='state_code',
            scope="usa",
            color='Pct of Colonies Impacted',
            hover_data=['State', 'Pct of Colonies Impacted'],
            color_continuous_scale=px.colors.sequential.YlOrRd,
            labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
            template='presentation',
            range_color=[0, 100]
        )

# App layout
app.layout = html.Div([
    html.H1("Varroa Mites Infect Bees Across USA", style={'textAlign': 'center', 'color': '#1f77b4'}),
    
    dcc.Dropdown(id="slct_year",
                 options=[
                     {"label": "2015", "value": 2015},
                     {"label": "2016", "value": 2016},
                     {"label": "2017", "value": 2017},
                     {"label": "2018", "value": 2018}],
                 multi=False,
                 value=2015,
                 style={'width': "40%", 'margin': '0 auto'}  # Center the dropdown
                 ),
    
    html.Div(id='output_container', children=[], style={'textAlign': 'center'}),
    html.Br(),
    
    dcc.Graph(id='Bees', figure=default_map_figure),  # Use the default map figure
    
    # Reset Button
    html.Button("Reset", id="reset_button", n_clicks=0, style={'margin': '10px auto'}),
])

# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='Bees', component_property='figure'),
     Output(component_id='slct_year', component_property='value')],
    [Input(component_id='slct_year', component_property='value'),
     Input(component_id='reset_button', component_property='n_clicks')]
)
def update_graph(option_slctd, reset_clicks):
    changed_id = [p['prop_id'] for p in callback_context.triggered][0]
    
    if 'reset_button' in changed_id:
        # Reset button was clicked, reset the dropdown and restore the default map
        return "Showing Data for 2015", default_map_figure, 2015  # Set the initial value and restore the default map
    else:
        # Dropdown selection was changed, update the map accordingly
        container = f"Showing Data for {option_slctd}"
    
        dff = df.copy()
        dff = dff[dff["Year"] == option_slctd]
        dff = dff[dff["Affected by"] == "Varroa_mites"]
    
        # Plotly Express
        fig = px.choropleth(
            data_frame=dff,
            locationmode='USA-states',
            locations='state_code',
            scope="usa",
            color='Pct of Colonies Impacted',
            hover_data=['State', 'Pct of Colonies Impacted'],
            color_continuous_scale=px.colors.sequential.YlOrRd,
            labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
            template='presentation',
            range_color=[0, 100]
        )
    
        return container, fig, option_slctd

if __name__ == '__main__':
     app.run_server(debug=True)


NameError: name 'option_slctd' is not defined